In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## План

- Подготовка данных для линейных моделей
  + Текст -> числа
  + Пропуски в данных
  + Категориальные переменные
- Обучение линейных моделей
- Регуляризация
- Масштабирование признаков
- Особенности SVM



In [ ]:
data = pd.read_csv("train.csv")

labels = data["Survived"]

columns_to_drop = ["PassengerId", "Survived", "Ticket", "Cabin", "Name"]
data.drop(columns_to_drop, axis=1, inplace=True)

### Преобразование текстовых признаков в числовые

Многие методы машинного обучения полагаются на то, что все признаки являются числами, которые можно складывать, вычитать и сравнивать. Поэтому важно преобразовать строковые признаки в числовые

Найдите какие признаки являются строковыми и преобразуйте их в числовые

In [ ]:
data.head(10)

In [ ]:
<ваш код>

In [ ]:
assert all(data.dtypes != object), "В датасете присутствуют строковые признаки!"

### Избавление от пропусков в данных

Также многие методы не умеют работать с пропусками в данных, когда значение признака у какого-то объекта равно NaN, как например возраст в 5 строчке. 

Для заполнения пропусков существует несколько способов:
- заполнить константой, например -1
- заполнить средним значением по колонке

Для первого способа удобно использовать метод fillna

In [ ]:
data.head(10)

In [ ]:
data.isnull().any()

In [ ]:
<ваш код>

In [ ]:
assert not data.isnull().any().any(), "В данных присутствуют пропуски!"

### Преобразование категориальных признаков: one-hot encoding

Еще одна проблема с признаками связана с категориальными переменными: город, страна, тип автомобиля. Методы машинного обучения полагаются на числовую природу признаков - что их можно складывать или вычитать. Однако даже если закодировать города Москва = 1, Санкт-Петербург = 2, Пермь = 3, то это не будет означать, что Москва + Санкт-Петербург = Пермь. 

С этим можно бороться с помощью преобразования 1 категориального признака в множество признаков-индикаторов, "Город==Москва?", "Город==Санкт-Петербург?".

In [ ]:
<ваш код>

## Линейные модели

### Линейная регрессия

$a(x) = w_1 * x_1 + w_2 * x_2 + \dots + w_n * x_n = <w, x>$ 

[sklearn.linear_model.LinearRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

### Линейный классификатор - логистическая регрессия
$a(x) = \sigma(<w, x>) = \dfrac{1}{1 + e^{-<w, x>}}$


[sklearn.linear_model.LogisticRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

### Линейный классификатор - SVM (Support Vector Machine)
$a(x) = sign(w_1 * x_1 + w_2 * x_2 + \dots + w_n * x_n) = sign(<w, x>)$ 


[sklearn.svm.SVC](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)

In [ ]:
model = ...

model.fit(data_encoded, labels)

In [ ]:
from sklearn.model_selection import cross_val_score

cross_val_score(model, data_encoded, labels, scoring="accuracy", cv=3).mean()

### Регуляризация

$a(x) = w_1 * x_1 + w_2 * x_2 + \dots + w_n * x_n = <w, x>$ 

Функция потерь:
\begin{equation}
L(x, y, w) = \sum\limits_{i = 1}^{n} \left(<w, x_i> - y_i\right)^2
\end{equation}
Чтобы модель меньше переобучалась, часто добавляют регуляризатор - дополнительный член в функцию потерь

\begin{equation}
L_2: L(x, y, w) = \sum\limits_{i = 1}^{n} \left(<w, x_i> - y_i\right)^2 + \sum\limits_{i=1}^{n} w_i^2 
\end{equation}
\begin{equation}
L_1: L(x, y, w) = \sum\limits_{i = 1}^{n} \left(<w, x_i> - y_i\right)^2 + \sum\limits_{i=1}^{n} |w_i|
\end{equation}

### Линейная регрессия: 

- Без регуляризации: [sklearn.linear_model.LinearRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)
- $L_1$ регуляризация: [sklearn.linear_model.Lasso](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)
- $L_2$ регуляризация: [sklearn.linear_model.Ridge](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html)

### Логистическая регрессия: 

Параметр `penalty` у [sklearn.linear_model.LogisticRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

In [ ]:
from sklearn.datasets import load_boston

X, y = load_boston(True)

In [ ]:
from sklearn.model_selection import cross_val_score

lr = ...

cross_val_score(lr, X, y, scoring="neg_mean_squared_error", cv=3).mean()

## Масштабирование признаков

При использовании регуляризации часто возникает такая проблема: если признаки имеют разный масштаб, то модель начинает обучаться гораздо хуже

### Методы масштабирования признаков

- нормализация в $\mathcal{N}(0, 1)$: [sklearn.preprocessing.scale](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.scale.html)
- MinMax масштабирование - в промежуток [0, 1]: [sklearn.preprocessing.minmax_scale](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.minmax_scale.html)
- MaxAbs масштабирование - в промежуток [-1, 1]: [sklearn.preprocessing.maxabs_scale](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.maxabs_scale.html)

In [ ]:
X_scaled = ...
print("Без масштабирования", cross_val_score(lr, X, y, scoring="neg_mean_squared_error", cv=3).mean())
print("C масштабированием", cross_val_score(lr, X_scaled, y, scoring="neg_mean_squared_error", cv=3).mean())

### Сила SVM - в ядрах!

Основное преимущество SVM заключается в том, что ядра позволяют решать нелинейные задачи также легко, как и линейные

Суть ядер заключается в том, чтобы перевести объекты выборки в новое пространство, где разделяющая плоскость будет уже линейной

![Image](http://www.eric-kim.net/eric-kim-net/posts/1/imgs/data_2d_to_3d.png)

In [ ]:
from sklearn.datasets import make_circles

X, y = make_circles(factor=0.5)
plt.scatter(X[:, 0], X[:, 1], c=y)

In [ ]:
print(cross_val_score(LogisticRegression(), X, y, scoring="accuracy", cv=3).mean())
print(cross_val_score(SVC(kernel="poly", degree=2), X, y, scoring="accuracy", cv=3).mean())

## Рекомендации
- Подготавливайте признаки для линейных моделей - так можно получить более качественную модель
- Следите за масштабом признаков и нормализуйте их - как в линейных моделях, так и в нейронных сетях
- Подбирайте оптимальный коэффициент регуляризации для снижения переобучения
- Используйте линейные модели в случае, если признаков очень много
- Используйте SVM с ядрами в случае нелинейных разделяющих поверхностей

## Заключение

Сегодня мы научились:
- Подготавливать признаки для линейных моделей
  - переводить строковые признаки в числовые
  - убирать пропуски
  - обрабатывать категориальные переменные
- Обучать различные линейные модели
  - SVM
  - линейная регрессия
  - логистическая регрессия
- Добавлять регуляризацию и масштабировать признаки